In [1]:
from vectorization.semantic_vector_database import SemanticVectorDatabase
from llms.providers import LLMRecommender
from generation.generator_simple import CodeSenseGenerator
from rag.rag import BasicContentRag

# Test requests
test_requests = [
    "I want to add a new rule to calculate loyalty points",
    "Create a new loyalty point reward for VIP customers",
    "Add validation for minimum order amount in loyalty rules",
    "Implement a service to check customer loyalty status",
    "Add integration with payment service for loyalty points"
]

generation_results = "results/generation"
def run_generator(vector: SemanticVectorDatabase, embedding_model: str, collection_name: str, recommender: LLMRecommender):
    """Demo function to test implementation generation"""

    collection = vector.get_collection(collection_name)
    rag = BasicContentRag(collection)

    generator = CodeSenseGenerator(recommender, rag)

    for sequence, request in enumerate(test_requests):
        print(f"\n{'='*80}")
        print(f"TESTING REQUEST: {request}")
        print(f"{'='*80}")

        try:
            # Generate suggestion
            coding_advice = generator.fetch_coding_advice(
                user_request=request
            )

            # Display results
            coding_advice.display()

            # Save suggestion
            coding_advice.save(f"{generation_results}/{embedding_model}/{collection_name}/test{sequence}.json")

        except Exception as e:
            print(f"Error testing {recommender}: {e}")

In [ ]:
import os
from dotenv import load_dotenv
from llms.anthropic import AnthropicRecommender
from llms.ollama import OllamaRecommender
from llms.openai import OpenAIRecommender
from vectorization.semantic_vector_database import SemanticVectorDatabase

load_dotenv()

vector_results = "results/vectorization"
rag_results = "results/rag"

chroma_base = "{model}/loyalty_points_kb"

loyalty_collection = "loyalty_code_semantics_{llm}"
anthropic = AnthropicRecommender(os.getenv("ANTHROPIC_API_KEY"))
ollama = OllamaRecommender()
openai = OpenAIRecommender(os.getenv("OPENAI_API_KEY"))

models = [
    {
        "model": "all-MiniLM-L6-v2",
        "llms": [ "claude3.5", "claude3.7", "claude4.0", "codellama", "gpt4.1"],
        "recommenders": [anthropic, ollama, openai]
    },
    {
        "model": "all-mpnet-base-v2",
        "llms": [ "claude3.5", "claude3.7", "claude4.0", "codellama", "gpt4.1"],
        "recommenders": [anthropic, ollama, openai]
    }
]

rag_testers = []
reports = []

for model in models:
    embedding_model = model["model"]
    print("embedding_model: ", embedding_model)
    db_path = f"{vector_results}/{chroma_base.format(model=embedding_model)}"
    print("db_path: ", db_path)

    for i, llm in enumerate(model["llms"]):
        collection_name = loyalty_collection.format(llm=llm)
        recommenders = model["recommenders"][i]

        vector_db = SemanticVectorDatabase(db_path, embedding_model)
        run_generator(vector_db, embedding_model, collection_name, recommenders)
